In [1]:
import pandas as pd
import pickle
import string
import matplotlib.pyplot as plt
import seaborn as sns
import tokenizer
import numpy as np
import pandas as pd
import pickle
import string
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
import re
import numpy as np
import multiprocessing 
from multiprocessing import Pool
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections
# Import required modules
import pandas as pd
from sklearn import preprocessing

# Set charts to view inline
%matplotlib inline

[nltk_data] Downloading package punkt to /home/dval/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
#read in hedonometer words from hedonometer.org and create dictionary of words and scores
file = pd.read_csv("Hedonometer.csv")
scores = file[["Word", "Happiness Score"]]
scores.columns = ["word", "score"]
scores = dict(zip(scores.word, scores.score))


In [14]:
#Using same uploaded file create dictionary of words and standard deviations of scores
sds = file[["Word", "Standard Deviation of Ratings"]]
sds.columns = ["word", "deviation"]
sds = dict(zip(sds.word, sds.deviation))

In [ ]:
#open kp tweets
with open('kp_tweets_cleaned_english.pickle', "rb") as f:
    tweets = pickle.load(f)

In [32]:
#Define formula for determining sentiment of a tweet
def sentiment(sentence):
    senti=0
    total=0
    words = sentence.split(" ")
    for word in words:
        if word in scores.keys():
            senti += scores[word]
            total += 1
    try:
        return senti/total
    except ZeroDivisionError:
        return 0

In [36]:
#Define formula applying sentiment formulat to dataframe
def return_sentiment(data):
    return pd.Series(data["clean_text"].apply(lambda x: sentiment(x)))

In [37]:
#Define formula to parallelize the code on multiple cores
def parallelize_dataframe(df, func, n_cores=25):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [42]:
#Run the hedonometer on the tweets
tweets["hedonometer"] = parallelize_dataframe(tweets, return_sentiment)

/home/dval/.conda/envs/symbolic/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [100]:
#Define function calcualting standard deviation of a tweets sentiment
def standard_deviation(sentence):
    standard_deviation=0
    total=0
    words = sentence.split(" ")
    for word in words:
        if word in sds.keys():
            standard_deviation += sds[word]
            total += 1
    try:
        return standard_deviation/total
    except ZeroDivisionError:
        return 0

In [85]:
#Define formula applying that to a dataframe
def return_standard_deviation(data):
    return pd.Series(data["clean_text"].apply(lambda x: standard_deviation(x)))

In [86]:
#Define formula to parallelize the code on multiple cores
def parallelize_dataframe(df, func, n_cores=25):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [87]:
#Calcualte the sd for each tweets sentiment
tweets["hedo_sd"] = parallelize_dataframe(tweets, return_standard_deviation)

/home/dval/.conda/envs/symbolic/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [92]:
#Define a formula to grab the words used to determine the sentiment of a tweet
def grab_hits(sentence):
    hits=[]
    words = sentence.split(" ")
    for word in words:
        if word in sds.keys():
            hits.append(word)
    return hits

In [94]:
#Define a forumula to apply this to a dataframe
def return_hits(data):
    return pd.Series(data["clean_text"].apply(lambda x: grab_hits(x)))

In [95]:
#Define a formula to parallelize function on multiple cores
def parallelize_dataframe(df, func, n_cores=25):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [ ]:
#Run funciton to pull the words from each tweet that determine sentiment into a new column
tweets["hits"] = parallelize_dataframe(tweets, return_hits)

Process ForkPoolWorker-157:
Process ForkPoolWorker-168:
Process ForkPoolWorker-151:
Process ForkPoolWorker-152:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/dval/.conda/envs/symbolic/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/dval/.conda/envs/symbolic/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/dval/.conda/envs/symbolic/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/dval/.conda/envs/symbolic/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/dval/.conda/envs/symbolic/lib/python3.5/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
Traceback (most recent call last):
Traceback (most recent call last):
  F

In [ ]:
#Save english language kp tweets with sentiments
with open("NEW_kp_hedonometer_english", "wb") as f:
    pickle.dump(tweets, f)
    
#WHEN DONE - REPEAT ON NON-KP ENGLISH TWEETS